In [1]:
# Load all the required libraries
import json
import pandas as pd
from IPython.display import display, HTML

In [2]:
def load_json_file(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

In [3]:
# Load different user schemas as json dictionaries
users_4me = load_json_file("./4me/users.schema.json")
users_scim_div = load_json_file("./scim-dev/users.json")
users_sf = load_json_file("./SF/users.json")
users_slack = load_json_file("./Slack/slack-users-schemas.json")

In [4]:
users_4me_attribute_list = users_4me["attributes"]
users_scim_div_attribute_list = users_scim_div["attributes"]
users_sf_attribute_list = users_sf["attributes"]
users_slack_attribute_list = users_slack["attributes"]

In [14]:
all_elements = set(users_4me_attribute_name_list).symmetric_difference(
    set(users_scim_div_attribute_name_list),
    set(users_sf_attribute_name_list),
    set(users_slack_attribute_name_list),
)

all_elements = sorted(all_elements)
print(all_elements)

TypeError: set.symmetric_difference() takes exactly one argument (3 given)

In [5]:
def get_attribute_name(attribute_list):
    attribute_name_list = []
    for attribute in attribute_list:
        attribute_name_list.append(attribute["name"])
    return sorted(attribute_name_list)

In [6]:
users_4me_attribute_name_list = get_attribute_name(users_4me_attribute_list)
users_scim_div_attribute_name_list = get_attribute_name(users_scim_div_attribute_list)
users_sf_attribute_name_list = get_attribute_name(users_sf_attribute_list)
users_slack_attribute_name_list = get_attribute_name(users_slack_attribute_list)

In [7]:
df_4me = pd.DataFrame(data=users_4me_attribute_name_list, columns=["4me"])
df_scim_div = pd.DataFrame(
    data=users_scim_div_attribute_name_list, columns=["scim.dev"]
)
df_sf = pd.DataFrame(data=users_sf_attribute_name_list, columns=["salesforce"])
df_slack = pd.DataFrame(data=users_slack_attribute_name_list, columns=["slack"])

df_merged_1 = pd.merge(
    df_4me, df_scim_div, how="outer", left_on="4me", right_on="scim.dev"
)
df_merged_2 = pd.merge(
    df_slack, df_sf, how="outer", left_on="slack", right_on="salesforce"
)
df_merged = pd.merge(
    df_merged_1, df_merged_2, how="outer", left_on="4me", right_on="slack"
)
df_merged.style.highlight_null(color="red")

,4me,scim.dev,slack,salesforce
0,active,active,active,active
1,addresses,addresses,addresses,addresses
2,displayName,displayName,displayName,displayName
3,emails,emails,emails,emails
4,entitlements,entitlements,nan,nan
5,groups,groups,groups,groups
6,nan,nan,id,id
7,ims,ims,nan,nan
8,locale,locale,locale,locale
9,name,name,name,name


## Notes:

The actual scim RFC does not have all those attribute related to users under regular users schemas.
Seems like a lot vender added a lot non-required attribute from enterprise users schemas to regular users schemas for unknown reason.


In [9]:
common_elements = (
    set(users_4me_attribute_name_list)
    & set(users_scim_div_attribute_name_list)
    & set(users_sf_attribute_name_list)
    & set(users_slack_attribute_name_list)
)
common_elements = sorted(common_elements)

for common_element in common_elements:
    for attribute in users_4me_attribute_list:
        if attribute["name"] == common_element:
            users_4me_attribute = attribute
    for attribute in users_scim_div_attribute_list:
        if attribute["name"] == common_element:
            users_scim_div_attribute = attribute
    for attribute in users_sf_attribute_list:
        if attribute["name"] == common_element:
            users_sf_attribute = attribute
    for attribute in users_slack_attribute_list:
        if attribute["name"] == common_element:
            users_slack_attribute = attribute
    print("=" * 100)
    print(f"Attribute: {common_element}")
    pd.set_option("display.max_colwidth", 30)
    pd.set_option("display.width", 1000)
    df_4me = pd.DataFrame(
        {"attribute": users_4me_attribute.keys(), "value": users_4me_attribute.values()}
    )
    df_scim_div = pd.DataFrame(
        {
            "attribute": users_scim_div_attribute.keys(),
            "value": users_scim_div_attribute.values(),
        }
    )
    df_sf = pd.DataFrame(
        {"attribute": users_sf_attribute.keys(), "value": users_sf_attribute.values()}
    )
    df_slack = pd.DataFrame(
        {
            "attribute": users_slack_attribute.keys(),
            "value": users_slack_attribute.values(),
        }
    )
    df_merged_1 = df_4me.merge(
        df_scim_div, how="outer", on="attribute", suffixes=("_4me", "_scim.dev")
    )
    df_merged_2 = df_sf.merge(
        df_slack, how="outer", on="attribute", suffixes=("_slack", "_salesforce")
    )
    df_merged = df_merged_1.merge(df_merged_2, how="outer", on="attribute")
    df_merged_styled = df_merged.style.highlight_null(color="red")
    # df_merged_styled = df_merged_styled.highl
    display(HTML(df_merged_styled.to_html()))
    # df_merged.style.highlight_null(color="red")
    # display(HTML(df_merged.to_html()))

Attribute: active


,attribute,value_4me,value_scim.dev,value_slack,value_salesforce
0,caseExact,nan,False,False,False
1,description,A Boolean value indicating the User's administrative status.,A Boolean value indicating the User's administrative status.,A Boolean value indicating the Users status,"A Boolean value indicating the User's administrative status. The definitive meaning of this attribute is determined by the Service Provider though a value of true infers the User is, for example, able to login while a value of false implies the User's account has been suspended."
2,multiValued,False,False,False,False
3,mutability,readWrite,readWrite,readWrite,readWrite
4,name,active,active,active,active
5,required,False,False,False,False
6,returned,default,default,nan,default
7,type,boolean,boolean,boolean,string
8,uniqueness,none,nan,nan,none


Attribute: addresses


,attribute,value_4me,value_scim.dev,value_slack,value_salesforce
0,caseExact,nan,False,False,False
1,description,A physical mailing address for this User.,A physical mailing address for this User.,The full mailing address,"A physical mailing address for this User. Canonical Type Values of work, home, and other. The value attribute is a complex type with the following sub-attributes. All Sub-Attributes are OPTIONAL."
2,multiValued,True,True,True,True
3,multiValuedAttributeChildName,nan,nan,nan,address
4,mutability,readWrite,readWrite,readWrite,readWrite
5,name,addresses,addresses,addresses,addresses
6,required,False,False,False,False
7,returned,default,default,nan,default
8,subAttributes,"[{'name': 'formatted', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'description': 'The full mailing address, formatted for display or use with a mailing label. This attribute MAY contain newlines.'}, {'name': 'streetAddress', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'description': 'The full street address component, which may include house number, street name, P.O. box, and multi-line extended street address information. This attribute MAY contain newlines.'}, {'name': 'locality', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'description': 'The city or locality component.'}, {'name': 'region', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'description': 'The state or region component.'}, {'name': 'postalCode', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'description': 'The zip code or postal code component.'}, {'name': 'country', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'description': 'The country name component.'}, {'name': 'type', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'canonicalValues': ['work', 'home', 'other'], 'description': ""A label indicating the attribute's function, e.g., 'work' or 'home'.""}]","[{'name': 'formatted', 'description': 'The full mailing address, formatted for display or use with a mailing label. This attribute MAY contain newlines.', 'type': 'string', 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'required': False, 'multiValued': False, 'caseExact': False}, {'name': 'streetAddress', 'description': 'The full street address component, which may include house number, street name, P.O. box, and multi-line extended street address information. This attribute MAY contain newlines.', 'type': 'string', 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'required': False, 'multiValued': False, 'caseExact': False}, {'name': 'locality', 'description': 'The city or locality component.', 'type': 'string', 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'required': False, 'multiValued': False, 'caseExact': False}, {'name': 'region', 'description': 'The state or region component.', 'type': 'string', 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'required': False, 'multiValued': False, 'caseExact': False}, {'name': 'postalCode', 'description': 'The zip code or postal code component.', 'type': 'string', 'mutability': 'readWrite', 'returned': 'default', 'uniqueness': 'none', 'required': False, 'multiValued': False, 'caseExact': False}, {'name': 'country', 'description': 'The country name component.',

Attribute: displayName


...

,attribute,value_4me,value_scim.dev,value_slack,value_salesforce
0,caseExact,False,False,False,False
1,description,"The user's title, such as 'Vice President.'","The user's title, such as ""Vice President"".","The Users title, such as Vice President","The user’s title, such as “Vice President”."
2,multiValued,False,False,False,False
3,mutability,readWrite,readWrite,readWrite,readWrite
4,name,title,title,title,title
5,required,False,False,False,False
6,returned,default,default,nan,default
7,type,string,string,string,string
8,uniqueness,none,none,nan,none


Attribute: userName


,attribute,value_4me,value_scim.dev,value_slack,value_salesforce
0,caseExact,False,False,False,False
1,description,"Unique identifier for the User, typically used by the user to directly authenticate to the service provider. Each User MUST include a non-empty userName value. This identifier MUST be unique across the service provider's entire set of Users.","Unique identifier for the User, typically used by the user to directly authenticate to the service provider. Each User MUST include a non-empty userName value. This identifier MUST be unique across the service provider's entire set of Users. REQUIRED.",Unique identifier for the User,"Unique identifier for the User, typically used by the user to directly authenticate to the service provider. Often displayed to the user as their unique identifier within the system (as opposed to id or externalId, which are generally opaque and not user-friendly identifiers). Each User MUST include a non-empty userName value. This identifier MUST be unique across the Service Consumer's entire set of Users. REQUIRED."
2,multiValued,False,False,False,False
3,mutability,readWrite,readWrite,readWrite,readWrite
4,name,userName,userName,userName,userName
5,required,True,True,True,True
6,returned,default,default,nan,default
7,type,string,string,string,string
8,uniqueness,server,server,nan,server


Attribute: userType


,attribute,value_4me,value_scim.dev,value_slack,value_salesforce
0,canonicalValues,nan,"['Contractor', 'Employee', 'Intern', 'Temp', 'External', 'Internal', 'Unknown']",nan,nan
1,caseExact,False,False,False,False
2,description,"Used to identify the relationship between the organization and the user. Typical values used might be 'Contractor', 'Employee', 'Intern', 'Temp', 'External', and 'Unknown', but any value may be used.","Used to identify the relationship between the organization and the user. Typical values used might be 'Contractor', 'Employee', 'Intern', 'Temp', 'External', and 'Unknown', but any value may be used.",The organization-to-user relationship,"Used to identify the organization to user relationship. Typical values used might be ""Contractor"", ""Employee"", ""Intern"", ""Temp"", ""External"", and ""Unknown"" but any value may be used."
3,multiValued,False,False,False,False
4,mutability,readWrite,readWrite,readWrite,readWrite
5,name,userType,userType,userType,userType
6,required,False,False,False,False
7,returned,default,default,nan,default
8,type,string,string,string,string
9,uniqueness,none,none,nan,none


In [ ]:
class ArbitraryPrecisionNumber:
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return str(self.value)

    def __add__(self, other):
        if isinstance(other, ArbitraryPrecisionNumber):
            return ArbitraryPrecisionNumber(self.value + other.value)
        elif isinstance(other, int):
            return ArbitraryPrecisionNumber(self.value + other)
        else:
            raise TypeError("Unsupported operand type for +")

    def __sub__(self, other):
        if isinstance(other, ArbitraryPrecisionNumber):
            return ArbitraryPrecisionNumber(self.value - other.value)
        elif isinstance(other, int):
            return ArbitraryPrecisionNumber(self.value - other)
        else:
            raise TypeError("Unsupported operand type for -")

    def __mul__(self, other):
        if isinstance(other, ArbitraryPrecisionNumber):
            return ArbitraryPrecisionNumber(self.value * other.value)
        elif isinstance(other, int):
            return ArbitraryPrecisionNumber(self.value * other)
        else:
            raise TypeError("Unsupported operand type for *")

    def __truediv__(self, other):
        if isinstance(other, ArbitraryPrecisionNumber):
            return ArbitraryPrecisionNumber(self.value / other.value)
        elif isinstance(other, int):
            return ArbitraryPrecisionNumber(self.value / other)
        else:
            raise TypeError("Unsupported operand type for /")